In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1035645,2021-02-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1035646,2021-02-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1035647,2021-02-28,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1035648,2021-03-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
28079,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28081,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28083,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28085,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28087,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
641362,2021-02-26,Arkansas,Arkansas,5001,1971,38.00,5000,Arkansas,AR,Arkansas,State,3017804
641364,2021-02-27,Arkansas,Arkansas,5001,1980,38.00,5000,Arkansas,AR,Arkansas,State,3017804
641366,2021-02-28,Arkansas,Arkansas,5001,1993,33.00,5000,Arkansas,AR,Arkansas,State,3017804
641368,2021-03-01,Arkansas,Arkansas,5001,1993,33.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1035645,2021-02-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1035646,2021-02-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1035647,2021-02-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1035648,2021-03-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-02') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
406,2021-03-02,Snohomish,Washington,31081,540.00,53061,WA,County,822083,3780.76,65.69
810,2021-03-02,Cook,Illinois,475386,9955.00,17031,IL,County,5150233,9230.38,193.29
1213,2021-03-02,Orange,California,261608,3952.00,6059,CA,County,3175692,8237.83,124.45
1615,2021-03-02,Maricopa,Arizona,511806,9157.00,4013,AZ,County,4485414,11410.45,204.15
2017,2021-03-02,Los Angeles,California,1194878,21554.00,6037,CA,County,10039107,11902.23,214.70
...,...,...,...,...,...,...,...,...,...,...,...
1035209,2021-03-02,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1035350,2021-03-02,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1035460,2021-03-02,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1035566,2021-03-02,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
406,2021-03-02,Snohomish,Washington,31081,540.00,53061,WA,County,822083,3780.76,65.69,65.69,3780.76
810,2021-03-02,Cook,Illinois,475386,9955.00,17031,IL,County,5150233,9230.38,193.29,193.29,9230.38
1213,2021-03-02,Orange,California,261608,3952.00,6059,CA,County,3175692,8237.83,124.45,124.45,8237.83
1615,2021-03-02,Maricopa,Arizona,511806,9157.00,4013,AZ,County,4485414,11410.45,204.15,204.15,11410.45
2017,2021-03-02,Los Angeles,California,1194878,21554.00,6037,CA,County,10039107,11902.23,214.70,214.70,11902.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035209,2021-03-02,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1035350,2021-03-02,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1035460,2021-03-02,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1035566,2021-03-02,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
729591,2021-03-02,Clarke,Alabama,3437,53.00,1025,AL,County,23622,14550.00,224.37,224.37,14550.00,1
670521,2021-03-02,Hale,Alabama,2106,68.00,1065,AL,County,14651,14374.45,464.13,464.13,14374.45,2
525081,2021-03-02,Lowndes,Alabama,1312,51.00,1085,AL,County,9726,13489.62,524.37,524.37,13489.62,3
465748,2021-03-02,Etowah,Alabama,13192,320.00,1055,AL,County,102268,12899.44,312.90,312.90,12899.44,4
391026,2021-03-02,Franklin,Alabama,3994,78.00,1059,AL,County,31362,12735.16,248.71,248.71,12735.16,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640013,2021-03-02,Converse,Wyoming,970,17.00,56009,WY,County,13822,7017.80,122.99,122.99,7017.80,19
640353,2021-03-02,Sublette,Wyoming,677,7.00,56035,WY,County,9831,6886.38,71.20,71.20,6886.38,20
884104,2021-03-02,Niobrara,Wyoming,149,2.00,56027,WY,County,2356,6324.28,84.89,84.89,6324.28,21
794918,2021-03-02,Lincoln,Wyoming,1241,11.00,56023,WY,County,19830,6258.19,55.47,55.47,6258.19,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
525081,2021-03-02,Lowndes,Alabama,1312,51.00,1085,AL,County,9726,13489.62,524.37,524.37,13489.62,3,1
670521,2021-03-02,Hale,Alabama,2106,68.00,1065,AL,County,14651,14374.45,464.13,464.13,14374.45,2,2
464376,2021-03-02,Clay,Alabama,1450,54.00,1027,AL,County,13235,10955.80,408.01,408.01,10955.80,21,3
223839,2021-03-02,Walker,Alabama,6538,258.00,1127,AL,County,63521,10292.66,406.16,406.16,10292.66,32,4
566118,2021-03-02,Greene,Alabama,882,32.00,1063,AL,County,8111,10874.12,394.53,394.53,10874.12,22,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564753,2021-03-02,Hot Springs,Wyoming,361,3.00,56017,WY,County,4413,8180.38,67.98,67.98,8180.38,17,19
775212,2021-03-02,Uinta,Wyoming,2062,12.00,56041,WY,County,20226,10194.80,59.33,59.33,10194.80,4,20
794918,2021-03-02,Lincoln,Wyoming,1241,11.00,56023,WY,County,19830,6258.19,55.47,55.47,6258.19,22,21
222792,2021-03-02,Teton,Wyoming,3359,9.00,56039,WY,County,23464,14315.55,38.36,38.36,14315.55,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.69,3780.76,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.69,3780.76,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.69,3780.76,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.69,3780.76,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.69,3780.76,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033202,2021-02-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1033203,2021-02-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1033204,2021-02-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1033205,2021-03-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
726813,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14550.00,39,1,2.00
726814,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14550.00,39,1,0.00
726815,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,224.37,14550.00,39,1,1.00
726816,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,224.37,14550.00,39,1,4.00
726817,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,224.37,14550.00,39,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808122,2021-02-26,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5511.60,7,23,0.00
808123,2021-02-27,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5511.60,7,23,0.00
808124,2021-02-28,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5511.60,7,23,0.00
808125,2021-03-01,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5511.60,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
522977,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13489.62,1,3,1.00,0.00
522978,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13489.62,1,3,0.00,0.00
522979,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13489.62,1,3,0.00,0.00
522980,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13489.62,1,3,0.00,0.00
522981,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13489.62,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521262,2021-02-26,Albany,Wyoming,3901,11.00,56001,WY,County,38880,10033.44,28.29,28.29,10059.16,23,6,7.00,0.00
521263,2021-02-27,Albany,Wyoming,3901,11.00,56001,WY,County,38880,10033.44,28.29,28.29,10059.16,23,6,0.00,0.00
521264,2021-02-28,Albany,Wyoming,3905,11.00,56001,WY,County,38880,10043.72,28.29,28.29,10059.16,23,6,4.00,0.00
521265,2021-03-01,Albany,Wyoming,3909,11.00,56001,WY,County,38880,10054.01,28.29,28.29,10059.16,23,6,4.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-02') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
267083,2021-03-02,Imperial,California,26913,635.00,6025,CA,County,181215,14851.42,350.41,350.41,14851.42,1,2,4.00,3.00
2017,2021-03-02,Los Angeles,California,1194878,21554.00,6037,CA,County,10039107,11902.23,214.70,214.70,11902.23,2,5,1347.00,87.00
467072,2021-03-02,Inyo,California,1285,37.00,6027,CA,County,18039,7123.45,205.11,205.11,7123.45,3,27,3.00,0.00
59685,2021-03-02,Stanislaus,California,56323,950.00,6099,CA,County,550660,10228.27,172.52,172.52,10228.27,4,10,88.00,1.00
73962,2021-03-02,Tulare,California,48086,766.00,6107,CA,County,466195,10314.57,164.31,164.31,10314.57,5,9,73.00,5.00
27717,2021-03-02,Riverside,California,290325,3829.00,6065,CA,County,2470546,11751.45,154.99,154.99,11751.45,6,6,552.00,37.00
47515,2021-03-02,San Joaquin,California,67040,1140.00,6077,CA,County,762148,8796.19,149.58,149.58,8796.19,7,18,211.00,14.00
423401,2021-03-02,Merced,California,29195,405.00,6047,CA,County,277680,10513.90,145.85,145.85,10513.90,8,8,48.00,4.00
26995,2021-03-02,Fresno,California,95677,1451.00,6019,CA,County,999101,9576.31,145.23,145.23,9576.31,9,14,129.00,8.00
566060,2021-03-02,Kings,California,22091,220.00,6031,CA,County,152940,14444.23,143.85,143.85,14444.23,10,3,9.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
984184,2021-03-02,Lassen,California,5601,19.00,6035,CA,County,30573,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00
267083,2021-03-02,Imperial,California,26913,635.00,6025,CA,County,181215,14851.42,350.41,350.41,14851.42,1,2,4.00,3.00
566060,2021-03-02,Kings,California,22091,220.00,6031,CA,County,152940,14444.23,143.85,143.85,14444.23,10,3,9.00,0.00
133223,2021-03-02,San Bernardino,California,286814,2952.00,6071,CA,County,2180085,13156.09,135.41,135.41,13156.09,12,4,59.00,12.00
2017,2021-03-02,Los Angeles,California,1194878,21554.00,6037,CA,County,10039107,11902.23,214.70,214.70,11902.23,2,5,1347.00,87.00
27717,2021-03-02,Riverside,California,290325,3829.00,6065,CA,County,2470546,11751.45,154.99,154.99,11751.45,6,6,552.00,37.00
167409,2021-03-02,Kern,California,103622,902.00,6029,CA,County,900202,11510.97,100.20,100.20,11510.97,19,7,200.00,25.00
423401,2021-03-02,Merced,California,29195,405.00,6047,CA,County,277680,10513.90,145.85,145.85,10513.90,8,8,48.00,4.00
73962,2021-03-02,Tulare,California,48086,766.00,6107,CA,County,466195,10314.57,164.31,164.31,10314.57,5,9,73.00,5.00
59685,2021-03-02,Stanislaus,California,56323,950.00,6099,CA,County,550660,10228.27,172.52,172.52,10228.27,4,10,88.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
267083,2021-03-02,Imperial,California,26913,635.00,6025,CA,County,181215,14851.42,350.41,350.41,14851.42,1,2,4.00,3.00
2017,2021-03-02,Los Angeles,California,1194878,21554.00,6037,CA,County,10039107,11902.23,214.70,214.70,11902.23,2,5,1347.00,87.00
467072,2021-03-02,Inyo,California,1285,37.00,6027,CA,County,18039,7123.45,205.11,205.11,7123.45,3,27,3.00,0.00
59685,2021-03-02,Stanislaus,California,56323,950.00,6099,CA,County,550660,10228.27,172.52,172.52,10228.27,4,10,88.00,1.00
73962,2021-03-02,Tulare,California,48086,766.00,6107,CA,County,466195,10314.57,164.31,164.31,10314.57,5,9,73.00,5.00
27717,2021-03-02,Riverside,California,290325,3829.00,6065,CA,County,2470546,11751.45,154.99,154.99,11751.45,6,6,552.00,37.00
47515,2021-03-02,San Joaquin,California,67040,1140.00,6077,CA,County,762148,8796.19,149.58,149.58,8796.19,7,18,211.00,14.00
423401,2021-03-02,Merced,California,29195,405.00,6047,CA,County,277680,10513.90,145.85,145.85,10513.90,8,8,48.00,4.00
26995,2021-03-02,Fresno,California,95677,1451.00,6019,CA,County,999101,9576.31,145.23,145.23,9576.31,9,14,129.00,8.00
566060,2021-03-02,Kings,California,22091,220.00,6031,CA,County,152940,14444.23,143.85,143.85,14444.23,10,3,9.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,350.41,14851.42,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,350.41,14851.42,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,350.41,14851.42,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,350.41,14851.42,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,350.41,14851.42,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3492,02/26/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00
3493,02/27/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00
3494,02/28/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00
3495,03/01/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,350.41,14851.42,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,350.41,14851.42,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,350.41,14851.42,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,350.41,14851.42,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,350.41,14851.42,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3566,02/26/21,Kings,22003,218.00,14386.69,142.54,143.85,14444.23,10,3,52.00,0.00
3567,02/27/21,Kings,22042,218.00,14412.19,142.54,143.85,14444.23,10,3,39.00,0.00
3568,02/28/21,Kings,22060,220.00,14423.96,143.85,143.85,14444.23,10,3,18.00,2.00
3569,03/01/21,Kings,22082,220.00,14438.34,143.85,143.85,14444.23,10,3,22.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)